In [2]:
import numpy as np
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')
from glob import glob
from tqdm import tqdm
import apt
from bs4 import BeautifulSoup
folder = ''
api = ''

In [3]:
!pip install PyPDF2 wget pypdf tabula-py pdfminer camelot-py[cv] -q
import tabula, camelot
import pypdf
import PyPDF2
import wget
from pypdf import PdfReader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 30.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.1 MB/s eta 0:00:00


### download table of drugs

In [ ]:
data_f = folder + 'data'
#url = 'https://www.ema.europa.eu/sites/default/files/Medicines_output_european_public_assessment_reports_en.xlsx'
#new url
url = 'https://www.ema.europa.eu/system/files/documents/other/medicines_output_european_public_assessment_reports_en.xlsx'
wget.download(url, out = data_f)

'/content/drive/MyDrive/pop_pharmacogenomics/onsides_intl/eu_drug_label/data/medicines_output_european_public_assessment_reports_en.xlsx'

### download individual drug files

In [ ]:
#download the table of all drug labels and their links first. read this in, and filter for human drugs
df = pd.read_excel(folder+'data/medicines_output_european_public_assessment_reports_en.xlsx', skiprows=8)
df = df[df.Category == 'Human']
print(df.shape[0])
df.head(1)

1759


,Category,Medicine name,Therapeutic area,International non-proprietary name (INN) / common name,Active substance,Product number,Patient safety,Authorisation status,ATC code,Additional monitoring,...,Vet pharmacotherapeutic group,Date of opinion,Decision date,Revision number,Condition / indication,Species,ATCvet code,First published,Revision date,URL
0,Human,Ryeqo,Leiomyoma,"relugolix, estradiol, norethisterone acetate","Relugolix, norethisterone acetate, estradiol h...",EMEA/H/C/005267,no,Authorised,H01CC54,yes,...,NaN,2021-07-16,2023-10-30 01:00:00,6.0,Ryeqo is indicated in adult women of reproduct...,NaN,NaN,2021-05-19 14:14:00,2023-12-04 18:41:00,https://www.ema.europa.eu/en/medicines/human/E...


In [ ]:
not_found = []
for i, row in tqdm(df.iterrows()):
  med = row['Medicine name']
  url = row['URL'].split('/')[-1]
  url = 'https://www.ema.europa.eu/en/documents/product-information/{}-epar-product-information_en.pdf'.format(url)
  try:
    wget.download(url, out = folder+'data/raw/{}_label.pdf'.format(str(med).lower()))
  except:
    not_found.append(med)

1759it [13:32,  2.17it/s]


In [ ]:
print('downloaded : {dl}, failed : {fail}, total : {tot}'.format(dl=str(df.shape[0]-len(not_found)), fail=len(not_found), tot=str(df.shape[0])))
print('retry download')

downloaded : 1588, failed : 171, total : 1759
retry download


In [ ]:
not_found = []
for med in tqdm(df['Medicine name']):
  url = 'https://www.ema.europa.eu/en/documents/product-information/{}-epar-product-information_en.pdf'.format(str(med).lower())
  try:
    wget.download(url, out = folder+'data/raw/{}_label.pdf'.format(str(med).lower()))
  except:
    not_found.append(med)

100%|██████████| 1759/1759 [18:29<00:00,  1.59it/s]


In [ ]:
print('downloaded : {dl}, failed : {fail}, total : {tot}'.format(dl=str(df.shape[0]-len(not_found)), fail=len(not_found), tot=str(df.shape[0])))
print('retry download')

downloaded : 1225, failed : 534, total : 1759
retry download


In [ ]:
not_found_2 = []
for med in tqdm(not_found):
  if 'known as' in med:
    med_alt = med.split(' (')[0].replace(' ','-').lower()
  else:
    med_alt = med.replace(' ','-').replace('(','').replace(')','').lower()
  url = 'https://www.ema.europa.eu/en/documents/product-information/{}-epar-product-information_en.pdf'.format(med_alt)
  try:
    wget.download(url, out = folder+'data/raw/{}_label.pdf'.format(str(med).lower()))
  except:
    not_found_2.append(med)

100%|██████████| 534/534 [04:02<00:00,  2.20it/s]


In [ ]:
len(not_found_2)

188

In [ ]:
not_found_3 = []
for med in tqdm(not_found_2):
  if 'previously' in med:
    med_alt = med.split(' (')[0].replace(' ','-').lower()
  else:
    med_alt = med.replace(' ','-').replace('(','').replace(')','').lower()
  url = 'https://www.ema.europa.eu/en/documents/product-information/{}-epar-product-information_en.pdf'.format(med_alt)
  try:
    wget.download(url, out = folder+'data/raw/{}_label.pdf'.format(str(med).lower()))
  except:
    not_found_3.append(med)

100%|██████████| 188/188 [00:36<00:00,  5.13it/s]


In [ ]:
df[df['Medicine name'].isin(not_found_3)]['Authorisation status'].value_counts()

Withdrawn     62
Refused       48
Authorised    38
Name: Authorisation status, dtype: int64

In [ ]:
a = df[df['Medicine name'].isin(not_found_3)]
a[a['Authorisation status'] == 'Authorised'].URL.tolist()[0]

'https://www.ema.europa.eu/en/medicines/human/EPAR/amlodipine-valsartan-mylan'

### extract pdf text into json

In [ ]:
!mkdir $folder'data/raw_txt'
!mkdir $folder'data/raw_tbl'

In [4]:
pdfs = glob(folder+'data/raw/*_label.pdf')
len(pdfs)

3229

In [ ]:
for pdf in tqdm(pdfs[0:]):
  drug = pdf.split('/')[-1].split('_label.pdf')[0]
  pdf_file = open(pdf, 'rb')
  read_pdf = PyPDF2.PdfReader(pdf_file)
  n_pages = len(read_pdf.pages)
  p_text = ''
  for n in range(n_pages):
    page = read_pdf.pages[n]
    page_content = page.extract_text()
    p_text += page_content
  txt_drug = folder+'data/raw_txt/{}.txt'.format(drug)
  with open(txt_drug, 'w+') as f:
    f.write(p_text)

100%|██████████| 58/58 [02:49<00:00,  2.92s/it]


In [ ]:
files = glob(folder+'data/raw/*.pdf')
for i, f in tqdm(enumerate(pdfs[700:])): #664~700
  try:
    tables = tabula.read_pdf(f,pages='all', silent=True)
    for i, table in enumerate(tables):
      table.to_csv(folder+'data/raw_tbl/{d}_{i}.csv'.format(d = f.split('/')[-1].split('_')[0], i = str(i)), index=False)
  except:
    continue

904it [3:05:17, 12.30s/it]


In [ ]:
txts = glob(folder+'data/raw_txt/*')
tbls = glob(folder+'data/raw_tbl/*')
len(txts), len(tbls)

(1603, 138910)

In [ ]:
len(list(set([i.split('/')[-1].split('_')[0] for i in tbls])))

1592